# 4. Training Framework


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このTraining Frameworkの部分について、プロジェクトの文脈に沿って説明させていただきます。

このコードは大きく5つの部分に分かれており、ソクラテス風チャットボットの学習過程を監視・評価するためのフレームワークを構築しています：

### 1. システムリソースの監視 (4.1)
```python
def log_memory_usage():
    """Log memory usage"""
```
- コンピュータのメモリ使用状況を監視します
- 学習中にメモリが不足しないよう見張る番人のような役割です

### 2. 評価指標の計算 (4.2)
```python
def compute_metrics(eval_preds):
```
- チャットボットの返答がどれだけ「ソクラテス風」になっているかを数値化します
- 例えば：
  - 「かね？」「だろうか？」などの問いかけ口調：60%の重み
  - 「君は」「君が」などの二人称の使用：25%の重み
  - 「では」「について」などの導入表現：15%の重み
- これらの要素を組み合わせて0〜1の間でスコアを算出します

### 3. 学習過程の記録 (4.3)
```python
class StyleCallback(TrainerCallback):
class TrainingMonitorCallback(TrainerCallback):
```
- 学習の進捗を監視し、記録します
- 例えば：
  - ソクラテス風スコアの推移
  - 学習の損失値（モデルの改善度合い）
  - 学習率の変化
- これらをグラフ化して視覚的に確認できるようにします

### 4. カスタム学習器 (4.4)
```python
class CustomTrainer(Trainer):
```
- 効率的な学習のための特別な設定を行います
- メモリの使用を最適化し、評価用のデータセットを適切なサイズに制限します

### 5. 学習設定 (4.5)
```python
training_args = TrainingArguments(
```
- 学習の具体的な設定を行います：
  - 30エポック（全データを30周学習）
  - 20ステップごとに評価
  - バッチサイズ4（一度に4つの対話を学習）
  - など

これは、例えば料理人が新しいレシピを開発する過程に似ています：
- 材料（学習データ）を使って料理（モデル）を作り
- 味見（評価）をしながら
- レシピ（パラメータ）を調整し
- その過程を記録して
- より良い料理（ソクラテス風の応答）ができるまで繰り返す

このフレームワークのおかげで、モデルがどれだけソクラテス風の話し方を身につけているか、常に監視しながら学習を進めることができます。

        
</div>
    </details>
</div>


# 4.1 System Resource Monitoring

In [ ]:
# 4.1 System Resource Monitoring
def log_memory_usage():
    """Log memory usage"""
    import psutil
    process = psutil.Process()
    logging.info(f"Memory usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")

def clear_memory():
    """Memory release function during training"""
    import gc
    gc.collect()
    torch.cuda.empty_cache()


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">


System Resource Monitoring（システムリソースの監視）について、より詳しく説明させていただきます。

### 概要
このコードは、ソクラテス風チャットボットの学習中にコンピュータのメモリ（記憶領域）の使用状況を監視・管理する2つの関数を定義しています。

### 1. メモリ使用量の記録 (`log_memory_usage()`)
```python
def log_memory_usage():
    import psutil
    process = psutil.Process()
    logging.info(f"Memory usage: {process.memory_info().rss / 1024 / 1024:.2f} MB")
```

これは「メモリの使用量を確認して記録する関数」です。

例えると、料理人（開発者）が冷蔵庫（メモリ）の中身をチェックするようなものです：
- どれくらいの材料（データ）が入っているか
- あとどれくらい材料を入れられるか
- 冷蔵庫が一杯になりそうな時は警告を出す

具体的には：
- 学習データ（ソクラテス風の対話データ）
- モデル自体（gemma-2b-jp）
- 計算途中の結果
などが、どれくらいメモリを使用しているかを監視します。

### 2. メモリの解放 (`clear_memory()`)
```python
def clear_memory():
    import gc
    gc.collect()
    torch.cuda.empty_cache()
```

これは「使い終わったメモリを掃除する関数」です。

例えると：
- 使い終わった食器（不要になったデータ）を洗って片付ける
- 調理台（作業スペース）を綺麗に掃除する
- 次の料理（学習）の準備をする

具体的には：
- 評価が終わった対話データ
- 計算済みの中間結果
- 一時的な変数
などの不要になったデータをメモリから削除します。

### なぜこれが必要か？

ソクラテス風チャットボットの学習では：
1. 大量の対話データを扱う
2. モデル自体も大きい
3. GPUでの計算も多い

そのため、メモリ管理が重要です：
- メモリ不足で学習が途中で止まるのを防ぐ
- 学習速度を維持する
- システムの安定性を確保する

例えば、1000組の対話データを学習させる場合：
- 各対話の処理後にメモリをチェック
- メモリ使用量が高くなったら掃除（解放）
- また次の対話の学習を続ける

このように、効率的なメモリ管理によって、長時間の学習を安定して行うことができます。


        
</div>
    </details>
</div>


# 4.2 Metrics Calculation System

In [ ]:
# 4.2 Metrics Calculation System
def compute_metrics(eval_preds):
    logits, labels = eval_preds  # Get logits and labels from eval_preds
    
    # Improve decoding process
    with torch.no_grad():
        logits = torch.tensor(logits).cpu()
        predictions = torch.argmax(logits, dim=-1)
        
        # Decode entire batch
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        
        # Add more detailed log output
        logging.info(f"Sample prediction: {decoded_preds[0][:100]}...")
        
        del logits, predictions  # Memory release
        torch.cuda.empty_cache()

        socratic_patterns = {
            'question_endings': ['かね', 'だろうか', 'ではないかね'],
            'address_patterns': ['君は', '君が', '君の'],
            'inquiry_leads': ['では', 'について']
        }

        def calculate_socratic_style(text):
            sentences = text.split('。')
            if not sentences:
                return 0.0
            
            scores = []
            for sent in sentences:
                if not sent.strip():
                    continue
                
                # Check if sentence ends with a question
                ends_with_question = any(sent.endswith(p) for p in socratic_patterns['question_endings'])
                # Proper use of second person
                uses_proper_address = any(p in sent for p in socratic_patterns['address_patterns'])
                # Use of inquiry introduction
                uses_inquiry_lead = any(p in sent for p in socratic_patterns['inquiry_leads'])
                
                # Sentence score (emphasize ending with a question)
                sentence_score = (
                    (ends_with_question * 0.6) +
                    (uses_proper_address * 0.25) +
                    (uses_inquiry_lead * 0.15)
                )
                scores.append(sentence_score)
            
            return np.mean(scores) if scores else 0.0

        style_scores = [calculate_socratic_style(pred) for pred in decoded_preds]
        final_score = np.mean(style_scores)
        
        return {
            'socratic_style': final_score,
        }


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



この部分は「ソクラテス風の話し方」をどれだけ習得できているかを数値化して評価するシステムです。詳しく説明していきます：

### 1. 評価の準備
```python
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    with torch.no_grad():
        logits = torch.tensor(logits).cpu()
        predictions = torch.argmax(logits, dim=-1)
```
- モデルの出力（数値の羅列）を実際の文章に変換する準備をします
- `with torch.no_grad():`は不要なメモリ使用を防ぐための設定です

### 2. ソクラテス風の特徴定義
```python
socratic_patterns = {
    'question_endings': ['かね', 'だろうか', 'ではないかね'],
    'address_patterns': ['君は', '君が', '君の'],
    'inquiry_leads': ['では', 'について']
}
```
ソクラテス風の話し方の3つの特徴を定義：
1. 質問の終わり方（例：「真実とは何かね？」）
2. 相手への呼びかけ方（例：「君はどう考えるか？」）
3. 問いかけの導入（例：「では、正義について考えよう」）

### 3. 採点システム
```python
def calculate_socratic_style(text):
    sentences = text.split('。')  # 文章を1文ずつに分割
    scores = []
    for sent in sentences:
        # 各特徴の確認
        ends_with_question = any(sent.endswith(p) for p in socratic_patterns['question_endings'])
        uses_proper_address = any(p in sent for p in socratic_patterns['address_patterns'])
        uses_inquiry_lead = any(p in sent for p in socratic_patterns['inquiry_leads'])
        
        # スコア計算（合計1.0になる重み付け）
        sentence_score = (
            (ends_with_question * 0.6) +  # 質問調の終わり方：60%
            (uses_proper_address * 0.25) + # 「君は」などの使用：25%
            (uses_inquiry_lead * 0.15)     # 「では」などの導入：15%
        )
        scores.append(sentence_score)
```

例えば、以下の応答をスコア化すると：

> 「では、君は正義について何を考えているのかね？」

- 「かね」で終わる → 0.6点
- 「君は」を使用 → 0.25点
- 「では」で導入 → 0.15点
- 合計：1.0点（満点！）

一方、以下の応答は：
> 「正義は大切です。」

- 質問調なし → 0点
- 「君は」なし → 0点
- 導入表現なし → 0点
- 合計：0点（ソクラテス風ではない）

### 4. 最終スコア計算
```python
style_scores = [calculate_socratic_style(pred) for pred in decoded_preds]
final_score = np.mean(style_scores)
```
- 全ての応答文のスコアを計算し
- その平均値を最終的な「ソクラテス度」として返します

このスコアが高いほど、モデルがソクラテス風の話し方を上手く学習できていることを示します。

        
</div>
    </details>
</div>


# 4.3 Training Callbacks

In [ ]:
# 4.3 Training Callbacks
# Custom callback modification
class StyleCallback(TrainerCallback):
    def __init__(self):
        self.socratic_scores = []
        
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if 'eval_socratic_style' in metrics:
            self.socratic_scores.append(metrics['eval_socratic_style'])
            
            # Log detailed information
            logging.info(f"Step {state.global_step}:")
            logging.info(f"Socratic Style Score: {metrics['eval_socratic_style']:.3f}")
    
    def on_train_end(self, args, state, control, **kwargs):
        # Log overall evaluation
        avg_score = sum(self.socratic_scores) / len(self.socratic_scores) if self.socratic_scores else 0
        
        logging.info("Training Complete!")
        logging.info(f"Average Socratic Style Score: {avg_score:.3f}")

# TrainingMonitorCallback also modified
class TrainingMonitorCallback(TrainerCallback):
    def __init__(self):
        self.train_start_time = None
        self.metrics_history = {
            'step': [],
            'socratic_style': [],  # Metric name changed
            'loss': [],
            'learning_rate': [],
            'epoch': []
        }
        self.output_dir = Path("model/training_progress")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
    def on_train_begin(self, args, state, control, **kwargs):
        self.train_start_time = datetime.now()
        log_memory_usage()
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        
        # Record metrics
        current_step = state.global_step
        
        # Record loss and learning_rate for all steps
        if 'loss' in logs:
            self.metrics_history['step'].append(current_step)
            self.metrics_history['epoch'].append(state.epoch)
            self.metrics_history['loss'].append(logs['loss'])
            self.metrics_history['learning_rate'].append(logs.get('learning_rate', None))
            self.metrics_history['socratic_style'].append(None)  # None for non-evaluation steps
        
        # Update socratic_style score in evaluation step
        if 'eval_socratic_style' in logs:
            # Update last entry (same step)
            if self.metrics_history['step'] and self.metrics_history['step'][-1] == current_step:
                self.metrics_history['socratic_style'][-1] = logs['eval_socratic_style']
            else:
                # Add new entry
                self.metrics_history['step'].append(current_step)
                self.metrics_history['epoch'].append(state.epoch)
                self.metrics_history['loss'].append(None)
                self.metrics_history['learning_rate'].append(None)
                self.metrics_history['socratic_style'].append(logs['eval_socratic_style'])
        
        # Save to CSV file
        df = pd.DataFrame(self.metrics_history)
        df.to_csv(self.output_dir / 'training_metrics.csv', index=False)
        
        # Update graph every 100 steps
        if current_step % 100 == 0:
            self._plot_metrics()
            
    def _plot_metrics(self):
        """Plot learning metrics and save"""
        plt.figure(figsize=(15, 8))
        
        # Plot Loss - Exclude None
        plt.subplot(2, 2, 1)
        valid_steps_loss = [s for s, v in zip(self.metrics_history['step'], self.metrics_history['loss']) if v is not None]
        valid_loss = [v for v in self.metrics_history['loss'] if v is not None]
        if valid_steps_loss:
            plt.plot(valid_steps_loss, valid_loss, label='Loss')
            plt.title('Training Loss')
            plt.xlabel('Step')
            plt.ylabel('Loss')
            plt.legend()
        
        # Plot Learning Rate - Exclude None
        plt.subplot(2, 2, 2)
        valid_steps_lr = [s for s, v in zip(self.metrics_history['step'], self.metrics_history['learning_rate']) if v is not None]
        valid_lr = [v for v in self.metrics_history['learning_rate'] if v is not None]
        if valid_steps_lr:
            plt.plot(valid_steps_lr, valid_lr, label='LR')
            plt.title('Learning Rate')
            plt.xlabel('Step')
            plt.ylabel('Learning Rate')
            plt.legend()
        
        # Plot Socratic Style Score - Exclude None
        plt.subplot(2, 2, 3)
        valid_steps = [s for s, v in zip(self.metrics_history['step'], self.metrics_history['socratic_style']) if v is not None]
        valid_scores = [v for v in self.metrics_history['socratic_style'] if v is not None]
        if valid_steps:
            plt.plot(valid_steps, valid_scores, label='Socratic Style')
            plt.title('Socratic Style Score')
            plt.xlabel('Step')
            plt.ylabel('Score')
            plt.legend()
        
        plt.tight_layout()
        plt.savefig(self.output_dir / 'training_progress.png')
        plt.close()
    
    def on_train_end(self, args, state, control, **kwargs):
        # Final learning result summary to save
        summary = {
            'training_duration': str(datetime.now() - self.train_start_time),
            'final_loss': self.metrics_history['loss'][-1] if self.metrics_history['loss'] else None,
            'best_socratic_score': max(filter(None, self.metrics_history['socratic_style'])) if self.metrics_history['socratic_style'] else None,
            'total_steps': len(self.metrics_history['step']),
            'final_epoch': self.metrics_history['epoch'][-1] if self.metrics_history['epoch'] else None
        }
        
        # Save summary as JSON file
        with open(self.output_dir / 'training_summary.json', 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
        
        # Final graph to save
        self._plot_metrics()
        
        logging.info("Training Complete!")
        logging.info(f"Training duration: {summary['training_duration']}")
        
        # None check added
        if summary['final_loss'] is not None:
            logging.info(f"Final loss: {summary['final_loss']:.4f}")
        else:
            logging.info("Final loss: Not available")
        
        if summary['best_socratic_score'] is not None:
            logging.info(f"Best Socratic style score: {summary['best_socratic_score']:.4f}")
        else:
            logging.info("Best Socratic style score: Not available")


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



この部分は、ソクラテス風チャットボットの学習過程を監視・記録するシステムです。2つの重要なクラスで構成されています：

### 1. StyleCallback クラス
```python
class StyleCallback(TrainerCallback):
```
これは「ソクラテス度」の監視係です。

例えば：
- モデルが「君は正義について何を考えているのかね？」と返答 → 高スコア
- モデルが「はい、そうですね」と返答 → 低スコア

主な機能：
- 評価時にソクラテス度スコアを記録
- 学習終了時に平均スコアを計算
- 結果をログに記録

### 2. TrainingMonitorCallback クラス
```python
class TrainingMonitorCallback(TrainerCallback):
```
これは「学習の進捗管理係」です。以下の情報を追跡・記録します：

1. **基本情報の記録**
```python
self.metrics_history = {
    'step': [],          # 学習ステップ
    'socratic_style': [], # ソクラテス度
    'loss': [],          # 損失値
    'learning_rate': [], # 学習率
    'epoch': []          # エポック
}
```

2. **グラフの作成** (`_plot_metrics`メソッド)
3つのグラフを作成：
- 損失値の推移（モデルの改善度）
- 学習率の変化
- ソクラテス度スコアの推移

例えば：
```
Step 100:
- 損失値: 2.5
- ソクラテス度: 0.3
（まだソクラテスらしくない）

Step 1000:
- 損失値: 1.2
- ソクラテス度: 0.7
（だいぶソクラテスらしくなってきた）

Step 2000:
- 損失値: 0.8
- ソクラテス度: 0.9
（かなりソクラテスらしい応答ができるように）
```

3. **最終結果のまとめ** (`on_train_end`メソッド)
```python
summary = {
    'training_duration': 学習にかかった時間,
    'final_loss': 最終的な損失値,
    'best_socratic_score': 最高のソクラテス度,
    'total_steps': 総ステップ数,
    'final_epoch': 最終エポック
}
```

これらの情報は：
- CSVファイルとして保存
- グラフとして視覚化
- JSONファイルとしてまとめられる

このシステムのおかげで：
1. 学習がうまくいっているか常に確認できる
2. モデルがどれだけソクラテスらしくなったか追跡できる
3. 問題があればすぐに気付ける
4. 学習結果を後で分析できる

例えるなら、料理人（開発者）が：
- 味の変化（ソクラテス度）を記録
- 火加減（学習率）を調整
- 調理時間（学習時間）を管理
- 最終的な出来栄えを評価
するようなものです。

        
</div>
    </details>
</div>


# 4.4 Custom Trainer Implementation

In [ ]:
# 4.4 Custom Trainer Implementation
class CustomTrainer(Trainer):
    def training_step(self, *args, **kwargs):
        loss = super().training_step(*args, **kwargs)
        if self.state.global_step % 100 == 0:
            clear_memory()
        return loss

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        if eval_dataset is not None:
            # Limit evaluation dataset to 100 samples
            eval_dataset = eval_dataset.select(range(min(100, len(eval_dataset))))
        return super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



このCustomTrainerクラスは、学習プロセスをカスタマイズするための特別な実装です。2つの重要な機能を持っています：

### 1. 学習ステップの管理 (`training_step`メソッド)
```python
def training_step(self, *args, **kwargs):
    loss = super().training_step(*args, **kwargs)
    if self.state.global_step % 100 == 0:
        clear_memory()
    return loss
```

これは「定期的な大掃除」のような機能です：

- 100ステップごとにメモリを掃除します
- 例えば：
  1. 100個の対話を学習
  2. メモリを掃除
  3. また100個の対話を学習
  4. メモリを掃除
  ...という具合

これにより：
- メモリ不足を防ぐ
- 学習の安定性を保つ
- システムの動作を快適に保つ

### 2. 評価の効率化 (`evaluate`メソッド)
```python
def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
    eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
    if eval_dataset is not None:
        # Limit evaluation dataset to 100 samples
        eval_dataset = eval_dataset.select(range(min(100, len(eval_dataset))))
    return super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)
```

これは「効率的な品質チェック」のような機能です：

- 評価用データを最大100サンプルに制限
- 例えば：
  - 全部で1000個の評価用対話データがあっても
  - 実際の評価には100個だけを使用
  - これでも十分な評価が可能

理由：
1. 評価が速くなる
2. メモリ使用量を抑えられる
3. それでも十分な精度で「ソクラテス度」を測定できる

具体例：
```
評価データ：
user: 「正義とは何でしょうか？」
model: 「君は正義とは何だと考えるかね？」

user: 「幸せについて考えています」
model: 「では、君にとって幸せとは何かね？」

...（このような対話を100個サンプリング）
```

このように、効率的な学習と評価を実現し、限られたリソースで最大の効果を得られるように工夫されています。

        
</div>
    </details>
</div>


# 4.5 Training Setup

In [ ]:

# 4.5 Training Setup
training_args = TrainingArguments(
    output_dir="./model",
    num_train_epochs=30,     
    learning_rate=8e-5,      
    weight_decay=0.06,         
    warmup_ratio=0.25,         
    lr_scheduler_type="cosine_with_restarts",  
    evaluation_strategy="steps",
    eval_steps=20,            
    save_strategy="steps",
    save_steps=20,
    gradient_accumulation_steps=8,   
    max_steps=-1,
    disable_tqdm=False,
    logging_dir="./model/logs",
    logging_strategy="steps",
    logging_steps=50,
    no_cuda=False,
    dataloader_num_workers=2,
    report_to=[],
    run_name=None,
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=2,  
    gradient_checkpointing=True,
    max_grad_norm=0.5,       
    dataloader_pin_memory=True,
    save_total_limit=3,
    fp16=True,
    optim="adamw_torch_fused",
    eval_accumulation_steps=8,
    load_best_model_at_end=True,
    metric_for_best_model="socratic_style",  
)


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">





この部分は、ソクラテス風チャットボットの学習に関する具体的な設定を定義しています。重要なパラメータを順に説明します：

### 1. 基本的な学習設定
```python
output_dir="./model",           # 学習結果の保存場所
num_train_epochs=30,           # 全データを30周学習
learning_rate=8e-5,            # 学習率（どれくらい大きく更新するか）
```

例えるなら：
- 30回同じ教材（データ）を繰り返し学習
- 少しずつ慎重に学習（小さな学習率）

### 2. 学習の進め方
```python
warmup_ratio=0.25,             # 準備運動期間（全体の25%）
lr_scheduler_type="cosine_with_restarts",  # 学習率の変化パターン
evaluation_strategy="steps",    # 20ステップごとに評価
eval_steps=20,
```

例えるなら：
1. まず準備運動（徐々に学習率を上げる）
2. 本格的な学習（最適な学習率で学習）
3. 定期的に「ソクラテス度」をチェック

### 3. バッチサイズ設定
```python
per_device_train_batch_size=4,  # 一度に学習する対話数
per_device_eval_batch_size=2,   # 一度に評価する対話数
```

具体例：
```
バッチ1:
user1: 「正義とは何でしょうか？」
model1: 「君は正義とは何だと考えるかね？」

user2: 「幸せについて考えています」
model2: 「では、君にとって幸せとは何かね？」

（このように4組の対話を一度に学習）
```

### 4. メモリと性能の最適化
```python
gradient_accumulation_steps=8,  # 8回分の計算をまとめて更新
gradient_checkpointing=True,    # メモリ効率を重視
fp16=True,                      # 16ビット精度で計算（メモリ節約）
```

これにより：
- メモリ使用を抑えつつ
- 効率的に学習を進められる

### 5. モデルの保存設定
```python
save_strategy="steps",          # 20ステップごとに保存
save_steps=20,
save_total_limit=3,            # 最新3つのモデルを保持
load_best_model_at_end=True,   # 最もソクラテスらしいモデルを採用
metric_for_best_model="socratic_style",  # ソクラテス度で評価
```

例えば：
1. 20ステップ目：ソクラテス度0.5のモデルを保存
2. 40ステップ目：ソクラテス度0.7のモデルを保存
3. 60ステップ目：ソクラテス度0.6のモデルを保存
→ 最終的にソクラテス度0.7のモデルを採用

このように、効率的かつ効果的な学習を実現するための詳細な設定が行われています。

        
</div>
    </details>
</div>
